### Metadata만 있는 데이터 생성

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime  # datetime 모듈 추가

class McDonaldsMenu:
    def __init__(self):
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"
        }
        self.menu_prices = {}

    def fetch_menu_data(self, sub_category_seq, page_num):
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }
        try:
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()
            return response.json().get('list', [])
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")
            return []

    def fetch_menu_detail(self, menu_code, sub_category_seq):
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            menu_name_tag = soup.find('h2', class_='ko')
            menu_desc_tag = soup.find('div', class_='desc')

            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            cleaned_menu_name = re.sub(r'\s+', '', menu_name)
            cleaned_menu_name = self.clean_text(cleaned_menu_name)

            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            menu_price = self.menu_prices.get(cleaned_menu_name, "가격 정보를 찾을 수 없음")

            if menu_price == "가격 정보를 찾을 수 없음":
                print(f"가격 정보 누락: {menu_name}")

            return {"menu_name": menu_name, "menu_desc": menu_desc, "menu_price": menu_price}
        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    def fetch_price_data(self):
        for cat_id in range(11, 16):
            if cat_id == 12:
                continue
            url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            script_content = soup.find_all('script', string=re.compile('ecommerce'))

            if not script_content:
                print(f"catId {cat_id}: 데이터를 포함한 스크립트를 찾지 못했습니다.")
            else:
                name_price_pattern = re.compile(r"'name'\s*:\s*\"(.*?)\".*?'price'\s*:\s*'(\d+)", re.DOTALL)

                for script in script_content:
                    matches = name_price_pattern.findall(script.string)
                    if matches:
                        for match in matches:
                            name, price = match
                            clean_name = re.sub(r'\s+', '', name)
                            clean_name = self.clean_text(clean_name)
                            self.menu_prices[clean_name] = price

    def fetch_nutrition_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_='tableType01 nutrDesc')

            if table:
                rows = table.find_all('tr')
                nutrition_info = [[col.text.strip() for col in row.find_all(['th', 'td'])] for row in rows]
                return nutrition_info
            else:
                return []
        except requests.exceptions.RequestException as e:
            print(f"영양 정보 API 요청 실패: {e}")
            return []

    def fetch_origin_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            origin_info_section = soup.find('ul', class_='origin_info')
            if origin_info_section:
                origin_info = origin_info_section.get_text(strip=True)
                # 괄호와 그 안의 내용을 제거합니다
                origin_info = re.sub(r'\([^)]*\)', '', origin_info)
                # 불필요한 공백을 제거합니다
                origin_info = re.sub(r'\s+', ' ', origin_info).strip()
                return re.sub(r'[™®]', '', origin_info)
            else:
                return "원산지 정보 없음"
        except requests.exceptions.RequestException as e:
            return "원산지 정보 요청 실패"

    def clean_text(self, text):
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)
        return cleaned_text

    def format_nutrition_info(self, nutrition_data):
        if not nutrition_data:
            return "영양소 정보 없음"
        headers = nutrition_data[0]
        values = nutrition_data[1]
        formatted_nutrition = {header: value for header, value in zip(headers, values) if header != "영양소"}
        # 모든 값을 문자열로 변환
        return {k: str(v) for k, v in formatted_nutrition.items()}

    def save_menus_to_file_and_return_json(self, all_menu_items_by_category):
        category_names = {
            1: "버거",
            7: "사이드",
            8: "디저트",
            9: "맥카페",
            10: "음료",
        }

        documents = []

        today = datetime.now().strftime("%m%d")
        file_path = f"/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_{today}_meta.json"
        with open(file_path, "w", encoding="utf-8") as f:
            for sub_category_seq, all_menu_items in all_menu_items_by_category.items():
                category_name = category_names.get(sub_category_seq, "기타")
                for item in all_menu_items:
                    kor_name = self.clean_text(item.get('kor_name', ''))
                    menu_code = item.get('seq', '')
                    menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)

                    nutrition_data = self.format_nutrition_info(self.fetch_nutrition_data(menu_code, sub_category_seq))
                    origin_info = self.fetch_origin_data(menu_code, sub_category_seq)

                    document = {
                        "metadata": {
                            "category": category_name,
                            "name": kor_name,
                            "description": menu_details["menu_desc"],
                            "price": str(menu_details["menu_price"]),  # 문자열로 변환
                            "nutrition": nutrition_data,
                            "origin_info": origin_info
                        }
                    }
                    documents.append(document)
                    
            json.dump(documents, f, ensure_ascii=False, indent=4)

        print("메뉴 데이터를 파일에 저장했습니다.")
        return json.dumps(documents, ensure_ascii=False, indent=4)


menu = McDonaldsMenu()
menu.fetch_price_data()

all_menu_items_by_category = {}
for sub_category_seq in [1, 7, 8, 9, 10]:
    all_menu_items = []
    for page_num in range(1, 6):
        menu_data = menu.fetch_menu_data(sub_category_seq, page_num)
        if not menu_data:
            break
        all_menu_items.extend(menu_data)
    all_menu_items_by_category[sub_category_seq] = all_menu_items

json_data = menu.save_menus_to_file_and_return_json(all_menu_items_by_category)

가격 정보 누락: 맥윙™
가격 정보 누락: 맥윙™콤보
가격 정보 누락: 골든 모짜렐라 치즈스틱
가격 정보 누락: 맥너겟®
가격 정보 누락: 맥스파이시®치킨 텐더
가격 정보 누락: 해쉬 브라운
가격 정보 누락: 딸기 오레오 맥플러리
가격 정보 누락: 초코 오레오 맥플러리
가격 정보 누락: 스트로베리콘
가격 정보 누락: 아이스크림콘
가격 정보 누락: 바닐라 선데이 아이스크림
가격 정보 누락: 초코 선데이 아이스크림
가격 정보 누락: 딸기 선데이 아이스크림
가격 정보 누락: 오레오 아포가토
가격 정보 누락: 아이스 카페라떼
가격 정보 누락: 디카페인 아이스 카페라떼
가격 정보 누락: 아이스 아메리카노
가격 정보 누락: 디카페인 아이스 아메리카노
가격 정보 누락: 아이스 드립 커피
가격 정보 누락: 바닐라 쉐이크 Medium
가격 정보 누락: 딸기 쉐이크 Medium
가격 정보 누락: 초코 쉐이크 Medium
가격 정보 누락: 오렌지 주스
메뉴 데이터를 파일에 저장했습니다.


### Pagecontent + Metadata 둘 다 빽빽

In [10]:
import requests
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime  # datetime 모듈 추가

class McDonaldsMenu:
    def __init__(self):
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"
        }
        self.menu_prices = {}

    def fetch_menu_data(self, sub_category_seq, page_num):
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }
        try:
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()
            return response.json().get('list', [])
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")
            return []

    def fetch_menu_detail(self, menu_code, sub_category_seq):
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            menu_name_tag = soup.find('h2', class_='ko')
            menu_desc_tag = soup.find('div', class_='desc')

            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            cleaned_menu_name = re.sub(r'\s+', '', menu_name)
            cleaned_menu_name = self.clean_text(cleaned_menu_name)

            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            menu_price = self.menu_prices.get(cleaned_menu_name, "가격 정보를 찾을 수 없음")

            if menu_price == "가격 정보를 찾을 수 없음":
                print(f"가격 정보 누락: {menu_name}")

            return {"menu_name": menu_name, "menu_desc": menu_desc, "menu_price": menu_price}
        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    def fetch_price_data(self):
        for cat_id in range(11, 16):
            if cat_id == 12:
                continue
            url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            script_content = soup.find_all('script', string=re.compile('ecommerce'))

            if not script_content:
                print(f"catId {cat_id}: 데이터를 포함한 스크립트를 찾지 못했습니다.")
            else:
                name_price_pattern = re.compile(r"'name'\s*:\s*\"(.*?)\".*?'price'\s*:\s*'(\d+)", re.DOTALL)

                for script in script_content:
                    matches = name_price_pattern.findall(script.string)
                    if matches:
                        for match in matches:
                            name, price = match
                            clean_name = re.sub(r'\s+', '', name)
                            clean_name = self.clean_text(clean_name)
                            self.menu_prices[clean_name] = price

    def fetch_nutrition_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_='tableType01 nutrDesc')

            if table:
                rows = table.find_all('tr')
                nutrition_info = [[col.text.strip() for col in row.find_all(['th', 'td'])] for row in rows]
                return nutrition_info
            else:
                return []
        except requests.exceptions.RequestException as e:
            print(f"영양 정보 API 요청 실패: {e}")
            return []

    def fetch_origin_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            origin_info_section = soup.find('ul', class_='origin_info')
            if origin_info_section:
                origin_info = origin_info_section.get_text(strip=True)
                # 괄호와 그 안의 내용을 제거합니다
                origin_info = re.sub(r'\([^)]*\)', '', origin_info)
                # 불필요한 공백을 제거합니다
                origin_info = re.sub(r'\s+', ' ', origin_info).strip()
                return re.sub(r'[™®]', '', origin_info)
            else:
                return "원산지 정보 없음"
        except requests.exceptions.RequestException as e:
            return "원산지 정보 요청 실패"

    def clean_text(self, text):
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)
        return cleaned_text

    def format_nutrition_info(self, nutrition_data):
        if not nutrition_data:
            return "영양소 정보 없음"
        headers = nutrition_data[0]
        values = nutrition_data[1]
        formatted_nutrition = {header: value for header, value in zip(headers, values) if header != "영양소"}
        # 모든 값을 문자열로 변환
        return {k: str(v) for k, v in formatted_nutrition.items()}

    def save_menus_to_file_and_return_json(self, all_menu_items_by_category):
        category_names = {
            1: "버거",
            7: "사이드",
            8: "디저트",
            9: "맥카페",
            10: "음료",
        }

        documents = []

        today = datetime.now().strftime("%m%d")
        file_path = f"/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_{today}_all.json"
        with open(file_path, "w", encoding="utf-8") as f:
            for sub_category_seq, all_menu_items in all_menu_items_by_category.items():
                category_name = category_names.get(sub_category_seq, "기타")
                for item in all_menu_items:
                    kor_name = self.clean_text(item.get('kor_name', ''))
                    menu_code = item.get('seq', '')
                    menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)

                    nutrition_data = self.format_nutrition_info(self.fetch_nutrition_data(menu_code, sub_category_seq))
                    origin_info = self.fetch_origin_data(menu_code, sub_category_seq)

                    document = {
                        "page_content": {
                            "name": kor_name,
                            "description": menu_details["menu_desc"],
                            "price": str(menu_details["menu_price"]),  # 문자열로 변환
                            "nutrition": nutrition_data,
                            "origin": origin_info
                        },
                        "metadata": {
                            "category": category_name,
                            "price": str(menu_details["menu_price"]),  # 문자열로 변환
                            "nutrition": nutrition_data,
                            "origin_info": origin_info
                        }
                    }
                    documents.append(document)

            json.dump(documents, f, ensure_ascii=False, indent=4)

        print("메뉴 데이터를 파일에 저장했습니다.")
        return json.dumps(documents, ensure_ascii=False, indent=4)


menu = McDonaldsMenu()
menu.fetch_price_data()

all_menu_items_by_category = {}
for sub_category_seq in [1, 7, 8, 9, 10]:
    all_menu_items = []
    for page_num in range(1, 6):
        menu_data = menu.fetch_menu_data(sub_category_seq, page_num)
        if not menu_data:
            break
        all_menu_items.extend(menu_data)
    all_menu_items_by_category[sub_category_seq] = all_menu_items

json_data = menu.save_menus_to_file_and_return_json(all_menu_items_by_category)

가격 정보 누락: 맥윙™
가격 정보 누락: 맥윙™콤보
가격 정보 누락: 골든 모짜렐라 치즈스틱
가격 정보 누락: 맥너겟®
가격 정보 누락: 맥스파이시®치킨 텐더
가격 정보 누락: 해쉬 브라운
가격 정보 누락: 딸기 오레오 맥플러리
가격 정보 누락: 초코 오레오 맥플러리
가격 정보 누락: 스트로베리콘
가격 정보 누락: 아이스크림콘
가격 정보 누락: 바닐라 선데이 아이스크림
가격 정보 누락: 초코 선데이 아이스크림
가격 정보 누락: 딸기 선데이 아이스크림
가격 정보 누락: 오레오 아포가토
가격 정보 누락: 아이스 카페라떼
가격 정보 누락: 디카페인 아이스 카페라떼
가격 정보 누락: 아이스 아메리카노
가격 정보 누락: 디카페인 아이스 아메리카노
가격 정보 누락: 아이스 드립 커피
가격 정보 누락: 바닐라 쉐이크 Medium
가격 정보 누락: 딸기 쉐이크 Medium
가격 정보 누락: 초코 쉐이크 Medium
가격 정보 누락: 오렌지 주스
메뉴 데이터를 파일에 저장했습니다.


### Pagecontent 간단 ver + Metadata

In [9]:
import requests
from bs4 import BeautifulSoup
import re
import json

class McDonaldsMenu:
    def __init__(self):
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"
        }
        self.menu_prices = {}

    def fetch_menu_data(self, sub_category_seq, page_num):
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }
        try:
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()
            return response.json().get('list', [])
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")
            return []

    def fetch_menu_detail(self, menu_code, sub_category_seq):
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            menu_name_tag = soup.find('h2', class_='ko')
            menu_desc_tag = soup.find('div', class_='desc')

            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            cleaned_menu_name = re.sub(r'\s+', '', menu_name)
            cleaned_menu_name = self.clean_text(cleaned_menu_name)

            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            menu_price = self.menu_prices.get(cleaned_menu_name, "가격 정보를 찾을 수 없음")

            if menu_price == "가격 정보를 찾을 수 없음":
                print(f"가격 정보 누락: {menu_name}")

            return {"menu_name": menu_name, "menu_desc": menu_desc, "menu_price": menu_price}
        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    def fetch_price_data(self):
        for cat_id in range(11, 16):
            if cat_id == 12:
                continue
            url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            script_content = soup.find_all('script', string=re.compile('ecommerce'))

            if not script_content:
                print(f"catId {cat_id}: 데이터를 포함한 스크립트를 찾지 못했습니다.")
            else:
                name_price_pattern = re.compile(r"'name'\s*:\s*\"(.*?)\".*?'price'\s*:\s*'(\d+)", re.DOTALL)

                for script in script_content:
                    matches = name_price_pattern.findall(script.string)
                    if matches:
                        for match in matches:
                            name, price = match
                            clean_name = re.sub(r'\s+', '', name)
                            clean_name = self.clean_text(clean_name)
                            self.menu_prices[clean_name] = price

    def fetch_nutrition_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_='tableType01 nutrDesc')

            if table:
                rows = table.find_all('tr')
                nutrition_info = [[col.text.strip() for col in row.find_all(['th', 'td'])] for row in rows]
                return nutrition_info
            else:
                return []
        except requests.exceptions.RequestException as e:
            print(f"영양 정보 API 요청 실패: {e}")
            return []

    def fetch_origin_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            origin_info_section = soup.find('ul', class_='origin_info')
            if origin_info_section:
                origin_info = origin_info_section.get_text(strip=True)
                return re.sub(r'[™®]', '', origin_info)
            else:
                return "원산지 정보 없음"
        except requests.exceptions.RequestException as e:
            return "원산지 정보 요청 실패"

    def clean_text(self, text):
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)
        return cleaned_text

    def format_nutrition_info(self, nutrition_data):
        if not nutrition_data:
            return "영양소 정보 없음"
        headers = nutrition_data[0]
        values = nutrition_data[1]
        formatted_nutrition = {header: value for header, value in zip(headers, values) if header != "영양소"}
        return formatted_nutrition

    def save_menus_to_file_and_return_json(self, all_menu_items_by_category):
        category_names = {
            1: "버거",
            7: "사이드",
            8: "디저트",
            9: "맥카페",
            10: "음료",
        }

        documents = []

        today = datetime.now().strftime("%m%d")
        file_path = f"/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_{today}_simple.json"
        with open(file_path, "w", encoding="utf-8") as f:
            for sub_category_seq, all_menu_items in all_menu_items_by_category.items():
                category_name = category_names.get(sub_category_seq, "기타")
                for item in all_menu_items:
                    kor_name = self.clean_text(item.get('kor_name', ''))
                    menu_code = item.get('seq', '')
                    menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)

                    nutrition_data = self.format_nutrition_info(self.fetch_nutrition_data(menu_code, sub_category_seq))
                    origin_info = self.fetch_origin_data(menu_code, sub_category_seq)

                    document = {
                        "page_content": f'{kor_name}{category_name}는 {menu_details["menu_desc"]}이고, 식재료는 {origin_info}가 사용되었고, 가격은 {menu_details["menu_price"]}원 입니다.',
                        "metadata": {
                            "category": category_name,
                            "name": kor_name,
                            "price": menu_details["menu_price"],
                            "nutrition": nutrition_data,
                            "origin_info": origin_info
                        }
                    }
                    documents.append(document)

            json.dump(documents, f, ensure_ascii=False, indent=4)

        print("메뉴 데이터를 파일에 저장했습니다.")
        return json.dumps(documents, ensure_ascii=False, indent=4)


menu = McDonaldsMenu()
menu.fetch_price_data()

all_menu_items_by_category = {}
for sub_category_seq in [1, 7, 8, 9, 10]:
    all_menu_items = []
    for page_num in range(1, 6):
        menu_data = menu.fetch_menu_data(sub_category_seq, page_num)
        if not menu_data:
            break
        all_menu_items.extend(menu_data)
    all_menu_items_by_category[sub_category_seq] = all_menu_items

json_data = menu.save_menus_to_file_and_return_json(all_menu_items_by_category)

가격 정보 누락: 맥윙™
가격 정보 누락: 맥윙™콤보
가격 정보 누락: 골든 모짜렐라 치즈스틱
가격 정보 누락: 맥너겟®
가격 정보 누락: 맥스파이시®치킨 텐더
가격 정보 누락: 해쉬 브라운
가격 정보 누락: 딸기 오레오 맥플러리
가격 정보 누락: 초코 오레오 맥플러리
가격 정보 누락: 스트로베리콘
가격 정보 누락: 아이스크림콘
가격 정보 누락: 바닐라 선데이 아이스크림
가격 정보 누락: 초코 선데이 아이스크림
가격 정보 누락: 딸기 선데이 아이스크림
가격 정보 누락: 오레오 아포가토
가격 정보 누락: 아이스 카페라떼
가격 정보 누락: 디카페인 아이스 카페라떼
가격 정보 누락: 아이스 아메리카노
가격 정보 누락: 디카페인 아이스 아메리카노
가격 정보 누락: 아이스 드립 커피
가격 정보 누락: 바닐라 쉐이크 Medium
가격 정보 누락: 딸기 쉐이크 Medium
가격 정보 누락: 초코 쉐이크 Medium
가격 정보 누락: 오렌지 주스
메뉴 데이터를 파일에 저장했습니다.


### Pagecontent no price + Metadata

In [11]:
import requests
from bs4 import BeautifulSoup
import re
import json

class McDonaldsMenu:
    def __init__(self):
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"
        }
        self.menu_prices = {}

    def fetch_menu_data(self, sub_category_seq, page_num):
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }
        try:
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()
            return response.json().get('list', [])
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")
            return []

    def fetch_menu_detail(self, menu_code, sub_category_seq):
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            menu_name_tag = soup.find('h2', class_='ko')
            menu_desc_tag = soup.find('div', class_='desc')

            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            cleaned_menu_name = re.sub(r'\s+', '', menu_name)
            cleaned_menu_name = self.clean_text(cleaned_menu_name)

            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            menu_price = self.menu_prices.get(cleaned_menu_name, "가격 정보를 찾을 수 없음")

            if menu_price == "가격 정보를 찾을 수 없음":
                print(f"가격 정보 누락: {menu_name}")

            return {"menu_name": menu_name, "menu_desc": menu_desc, "menu_price": menu_price}
        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    def fetch_price_data(self):
        for cat_id in range(11, 16):
            if cat_id == 12:
                continue
            url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            script_content = soup.find_all('script', string=re.compile('ecommerce'))

            if not script_content:
                print(f"catId {cat_id}: 데이터를 포함한 스크립트를 찾지 못했습니다.")
            else:
                name_price_pattern = re.compile(r"'name'\s*:\s*\"(.*?)\".*?'price'\s*:\s*'(\d+)", re.DOTALL)

                for script in script_content:
                    matches = name_price_pattern.findall(script.string)
                    if matches:
                        for match in matches:
                            name, price = match
                            clean_name = re.sub(r'\s+', '', name)
                            clean_name = self.clean_text(clean_name)
                            self.menu_prices[clean_name] = price

    def fetch_nutrition_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_='tableType01 nutrDesc')

            if table:
                rows = table.find_all('tr')
                nutrition_info = [[col.text.strip() for col in row.find_all(['th', 'td'])] for row in rows]
                return nutrition_info
            else:
                return []
        except requests.exceptions.RequestException as e:
            print(f"영양 정보 API 요청 실패: {e}")
            return []

    def fetch_origin_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            origin_info_section = soup.find('ul', class_='origin_info')
            if origin_info_section:
                origin_info = origin_info_section.get_text(strip=True)
                return re.sub(r'[™®]', '', origin_info)
            else:
                return "원산지 정보 없음"
        except requests.exceptions.RequestException as e:
            return "원산지 정보 요청 실패"

    def clean_text(self, text):
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)
        return cleaned_text

    def format_nutrition_info(self, nutrition_data):
        if not nutrition_data:
            return "영양소 정보 없음"
        headers = nutrition_data[0]
        values = nutrition_data[1]
        formatted_nutrition = {header: value for header, value in zip(headers, values) if header != "영양소"}
        return formatted_nutrition

    def save_menus_to_file_and_return_json(self, all_menu_items_by_category):
        category_names = {
            1: "버거",
            7: "사이드",
            8: "디저트",
            9: "맥카페",
            10: "음료",
        }

        documents = []

        today = datetime.now().strftime("%m%d")
        file_path = f"/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_{today}_no_price.json"
        with open(file_path, "w", encoding="utf-8") as f:
            for sub_category_seq, all_menu_items in all_menu_items_by_category.items():
                category_name = category_names.get(sub_category_seq, "기타")
                for item in all_menu_items:
                    kor_name = self.clean_text(item.get('kor_name', ''))
                    menu_code = item.get('seq', '')
                    menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)

                    nutrition_data = self.format_nutrition_info(self.fetch_nutrition_data(menu_code, sub_category_seq))
                    origin_info = self.fetch_origin_data(menu_code, sub_category_seq)

                    document = {
                        "page_content": f'{kor_name}{category_name}는 {menu_details["menu_desc"]}이고, 식재료는 {origin_info}가 사용되었습니다.',
                        "metadata": {
                            "category": category_name,
                            "name": kor_name,
                            "price": menu_details["menu_price"],
                            "nutrition": nutrition_data,
                            "origin_info": origin_info
                        }
                    }
                    documents.append(document)

            json.dump(documents, f, ensure_ascii=False, indent=4)

        print("메뉴 데이터를 파일에 저장했습니다.")
        return json.dumps(documents, ensure_ascii=False, indent=4)


menu = McDonaldsMenu()
menu.fetch_price_data()

all_menu_items_by_category = {}
for sub_category_seq in [1, 7, 8, 9, 10]:
    all_menu_items = []
    for page_num in range(1, 6):
        menu_data = menu.fetch_menu_data(sub_category_seq, page_num)
        if not menu_data:
            break
        all_menu_items.extend(menu_data)
    all_menu_items_by_category[sub_category_seq] = all_menu_items

json_data = menu.save_menus_to_file_and_return_json(all_menu_items_by_category)

가격 정보 누락: 맥윙™
가격 정보 누락: 맥윙™콤보
가격 정보 누락: 골든 모짜렐라 치즈스틱
가격 정보 누락: 맥너겟®
가격 정보 누락: 맥스파이시®치킨 텐더
가격 정보 누락: 해쉬 브라운
가격 정보 누락: 딸기 오레오 맥플러리
가격 정보 누락: 초코 오레오 맥플러리
가격 정보 누락: 스트로베리콘
가격 정보 누락: 아이스크림콘
가격 정보 누락: 바닐라 선데이 아이스크림
가격 정보 누락: 초코 선데이 아이스크림
가격 정보 누락: 딸기 선데이 아이스크림
가격 정보 누락: 오레오 아포가토
가격 정보 누락: 아이스 카페라떼
가격 정보 누락: 디카페인 아이스 카페라떼
가격 정보 누락: 아이스 아메리카노
가격 정보 누락: 디카페인 아이스 아메리카노
가격 정보 누락: 아이스 드립 커피
가격 정보 누락: 바닐라 쉐이크 Medium
가격 정보 누락: 딸기 쉐이크 Medium
가격 정보 누락: 초코 쉐이크 Medium
가격 정보 누락: 오렌지 주스
메뉴 데이터를 파일에 저장했습니다.


### Pagecontent만(no metadata)

In [12]:
import requests
from bs4 import BeautifulSoup
import re
import json

class McDonaldsMenu:
    def __init__(self):
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"
        }
        self.menu_prices = {}

    def fetch_menu_data(self, sub_category_seq, page_num):
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }
        try:
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()
            return response.json().get('list', [])
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")
            return []

    def fetch_menu_detail(self, menu_code, sub_category_seq):
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            menu_name_tag = soup.find('h2', class_='ko')
            menu_desc_tag = soup.find('div', class_='desc')

            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            cleaned_menu_name = re.sub(r'\s+', '', menu_name)
            cleaned_menu_name = self.clean_text(cleaned_menu_name)

            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            menu_price = self.menu_prices.get(cleaned_menu_name, "가격 정보를 찾을 수 없음")

            if menu_price == "가격 정보를 찾을 수 없음":
                print(f"가격 정보 누락: {menu_name}")

            return {"menu_name": menu_name, "menu_desc": menu_desc, "menu_price": menu_price}
        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    def fetch_price_data(self):
        for cat_id in range(11, 16):
            if cat_id == 12:
                continue
            url = f"https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId={cat_id}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            script_content = soup.find_all('script', string=re.compile('ecommerce'))

            if not script_content:
                print(f"catId {cat_id}: 데이터를 포함한 스크립트를 찾지 못했습니다.")
            else:
                name_price_pattern = re.compile(r"'name'\s*:\s*\"(.*?)\".*?'price'\s*:\s*'(\d+)", re.DOTALL)

                for script in script_content:
                    matches = name_price_pattern.findall(script.string)
                    if matches:
                        for match in matches:
                            name, price = match
                            clean_name = re.sub(r'\s+', '', name)
                            clean_name = self.clean_text(clean_name)
                            self.menu_prices[clean_name] = price

    def fetch_nutrition_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_='tableType01 nutrDesc')

            if table:
                rows = table.find_all('tr')
                nutrition_info = [[col.text.strip() for col in row.find_all(['th', 'td'])] for row in rows]
                return nutrition_info
            else:
                return []
        except requests.exceptions.RequestException as e:
            print(f"영양 정보 API 요청 실패: {e}")
            return []

    def fetch_origin_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            origin_info_section = soup.find('ul', class_='origin_info')
            if origin_info_section:
                origin_info = origin_info_section.get_text(strip=True)
                return re.sub(r'[™®]', '', origin_info)
            else:
                return "원산지 정보 없음"
        except requests.exceptions.RequestException as e:
            return "원산지 정보 요청 실패"

    def clean_text(self, text):
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)
        return cleaned_text

    def format_nutrition_info(self, nutrition_data):
        if not nutrition_data:
            return "영양소 정보 없음"
        headers = nutrition_data[0]
        values = nutrition_data[1]
        formatted_nutrition = {header: value for header, value in zip(headers, values) if header != "영양소"}
        return formatted_nutrition

    def save_menus_to_file_and_return_json(self, all_menu_items_by_category):
        category_names = {
            1: "버거",
            7: "사이드",
            8: "디저트",
            9: "맥카페",
            10: "음료",
        }

        documents = []

        today = datetime.now().strftime("%m%d")
        file_path = f"/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_{today}.json"
        with open(file_path, "w", encoding="utf-8") as f:
            for sub_category_seq, all_menu_items in all_menu_items_by_category.items():
                category_name = category_names.get(sub_category_seq, "기타")
                for item in all_menu_items:
                    kor_name = self.clean_text(item.get('kor_name', ''))
                    menu_code = item.get('seq', '')
                    menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)

                    nutrition_data = self.format_nutrition_info(self.fetch_nutrition_data(menu_code, sub_category_seq))
                    origin_info = self.fetch_origin_data(menu_code, sub_category_seq)

                    document = {
                        "page_content": {
                            "name": kor_name,
                            "category": category_name,
                            "description": menu_details["menu_desc"],
                            "price": str(menu_details["menu_price"]),  # 문자열로 변환
                            "nutrition": nutrition_data,
                            "origin": origin_info
                        }
                    }
                    documents.append(document)

            json.dump(documents, f, ensure_ascii=False, indent=4)

        print("메뉴 데이터를 파일에 저장했습니다.")
        return json.dumps(documents, ensure_ascii=False, indent=4)


menu = McDonaldsMenu()
menu.fetch_price_data()

all_menu_items_by_category = {}
for sub_category_seq in [1, 7, 8, 9, 10]:
    all_menu_items = []
    for page_num in range(1, 6):
        menu_data = menu.fetch_menu_data(sub_category_seq, page_num)
        if not menu_data:
            break
        all_menu_items.extend(menu_data)
    all_menu_items_by_category[sub_category_seq] = all_menu_items

json_data = menu.save_menus_to_file_and_return_json(all_menu_items_by_category)

가격 정보 누락: 맥윙™
가격 정보 누락: 맥윙™콤보
가격 정보 누락: 골든 모짜렐라 치즈스틱
가격 정보 누락: 맥너겟®
가격 정보 누락: 맥스파이시®치킨 텐더
가격 정보 누락: 해쉬 브라운
가격 정보 누락: 딸기 오레오 맥플러리
가격 정보 누락: 초코 오레오 맥플러리
가격 정보 누락: 스트로베리콘
가격 정보 누락: 아이스크림콘
가격 정보 누락: 바닐라 선데이 아이스크림
가격 정보 누락: 초코 선데이 아이스크림
가격 정보 누락: 딸기 선데이 아이스크림
가격 정보 누락: 오레오 아포가토
가격 정보 누락: 아이스 카페라떼
가격 정보 누락: 디카페인 아이스 카페라떼
가격 정보 누락: 아이스 아메리카노
가격 정보 누락: 디카페인 아이스 아메리카노
가격 정보 누락: 아이스 드립 커피
가격 정보 누락: 바닐라 쉐이크 Medium
가격 정보 누락: 딸기 쉐이크 Medium
가격 정보 누락: 초코 쉐이크 Medium
가격 정보 누락: 오렌지 주스
메뉴 데이터를 파일에 저장했습니다.
